In [1]:
import numpy as np

# generate data

In [2]:
# In real world, you cannot learn how the data was generated. So do not rely on this function when coding your lab.
def generate_data(dim, num):
    x = np.random.normal(0, 10, [num, dim])
    coef = np.random.uniform(-1, 1, [dim, 1])
    pred = np.dot(x, coef)
    pred_n = (pred - np.mean(pred)) / np.sqrt(np.var(pred))
    label = np.sign(pred_n)
    mislabel_value = np.random.uniform(0, 1, num)
    mislabel = 0
    for i in range(num):
        if np.abs(pred_n[i]) < 1 and mislabel_value[i] > 0.9 + 0.1 * np.abs(pred_n[i]):
            label[i] *= -1
            mislabel += 1
    return x, label, mislabel/num

In [3]:
# example
x, y, mr = generate_data(5, 100)

In [4]:
x[:5], y[:5]

(array([[  2.80909779, -12.49076017,   1.67531333,  -2.08100856,
          16.76514414],
        [  7.28801727,  14.05095341, -12.82720853,  -5.4981614 ,
           3.66341354],
        [ 14.09719883,  -4.43475596,  -0.09525357,   3.7283802 ,
         -10.77734169],
        [  9.29088901,   3.30030333, -12.01920321,   8.39161191,
          -3.32216729],
        [ -2.98855415,   0.3067418 , -12.46454395,  15.8440152 ,
          -5.91963409]]),
 array([[ 1.],
        [-1.],
        [ 1.],
        [ 1.],
        [ 1.]]))

In [5]:
y=y.flatten()
print(y)
print(mr)

[ 1. -1.  1.  1.  1.  1. -1. -1.  1. -1.  1. -1.  1. -1. -1.  1.  1.  1.
 -1. -1. -1. -1.  1. -1.  1. -1.  1.  1. -1.  1.  1. -1.  1.  1. -1.  1.
  1. -1.  1.  1.  1. -1. -1.  1. -1. -1.  1.  1.  1.  1.  1.  1. -1. -1.
 -1.  1.  1.  1.  1.  1. -1.  1.  1. -1.  1. -1.  1.  1.  1. -1. -1.  1.
  1.  1.  1. -1. -1.  1. -1.  1.  1.  1. -1.  1.  1. -1.  1. -1. -1.  1.
  1. -1.  1.  1. -1.  1.  1.  1. -1. -1.]
0.03


# write your model class

In [6]:
import random

# you can do anything necessary about the model
class SVM1:
    def __init__(self, dim, max_iter, C, tol, epsilon):
        """
        You can add some other parameters, which I think is not necessary
        """
        self.dim = dim
        self.max_iter = max_iter
        self.C = C
        self.tol = tol
        self.epsilon = epsilon

    def fit(self, X, y):
        """
        Fit the coefficients via your methods
        """
        self.b = 0
        num = X.shape[0]
        alpha = np.zeros(num)
        for _ in range(self.max_iter):
            for i in range(num):
                self.w = np.dot(X.T,np.multiply(alpha,y))

                gi = np.sign(np.dot(self.w.T, X[i,:].T)+self.b).astype(int)
                # gi = np.dot(self.w.T, self.X[i,:].T)+self.b
                Ei = gi - y[i]
                if (y[i]*Ei<-self.tol and alpha[i]<self.C) or (y[i]*Ei>self.tol and alpha[i]>0):
                    # random select(except i)
                    l = list(range(num))
                    j = random.choice(l[:i] + l[i+1:])

                    Kij = np.dot(X[i,:], X[i,:].T)+np.dot(X[j,:], X[j,:].T)-2*np.dot(X[i,:], X[j,:].T)
                    if Kij <= 0:
                        continue

                    gj = np.sign(np.dot(self.w.T, X[j,:].T)+self.b).astype(int)
                    # gj = np.dot(self.w.T,self.X[j,:].T)+self.b
                    Ej = gj - y[j]

                    alpha_old_i,alpha_old_j = alpha[i],alpha[j]

                    if y[i] != y[j]:
                        L = max(0,alpha_old_j-alpha_old_i)
                        H = min(self.C, self.C+alpha_old_j-alpha_old_i)
                    else:
                        L = max(0,alpha_old_i+alpha_old_j-self.C)
                        H = min(self.C,alpha_old_i+alpha_old_j)

                    # 定义域确定
                    if abs(L - H) < self.epsilon:
                        continue

                    alpha[j] = alpha_old_j + float(y[j]*(Ei-Ej))/Kij
                    alpha[j] = max(alpha[j],L)
                    alpha[j] = min(alpha[j],H)

                    alpha[i] = alpha_old_i + y[i]*y[j]*(alpha_old_j - alpha[j])
                    if abs(alpha[j] - alpha_old_j) < self.epsilon and abs(alpha[i]-alpha_old_i)<self.epsilon:
                        continue

                    # bi = self.b - Ei - y[i] * (alpha[i]-alpha_old_i)*np.dot(self.X[i,:],self.X[i,:].T) - y[j]*(alpha[j]-alpha_old_j)*np.dot(self.X[j,:],self.X[i,:].T)
                    #
                    # if (0 < alpha[i]) and (self.C > alpha[i]):
                    #     self.b = bi
                    #     continue
                    #
                    # bj = self.b - Ej - y[i] * (alpha[i]-alpha_old_i)*np.dot(self.X[i,:],self.X[j,:].T) - y[j]*(alpha[j]-alpha_old_j)*np.dot(self.X[j,:],self.X[j,:].T)
                    #
                    # if (0 < alpha[j]) and (self.C > alpha[j]):
                    #     self.b = bj
                    #     continue
                    # else:
                    #     self.b = (bi+bj)/2.0
                    #

                    S = np.where((0 < alpha) & (alpha< self.C), 1, 0)
                    if np.sum(S):
                        b_pre = np.dot(X.T,np.multiply(alpha,y))
                        self.b = np.dot(y-np.dot(b_pre,X.T),S)/np.sum(S)

        self.w = np.dot(X.T,np.multiply(alpha,y))

    def predict(self, X):
        """
        Use the trained model to generate prediction probabilities on a new
        collection of data points.
        """
        output = np.dot(X, self.w) + self.b
        predicted_label = np.sign(output)
        return predicted_label


In [7]:
# you can do anything necessary about the model
class SVM2:
    def __init__(self, dim, lr, max_iter, C):
        """
        You can add some other parameters, which I think is not necessary
        """
        self.dim = dim
        self.lr = lr
        self.max_iter = max_iter
        self.C = C

    def fit(self, X, y):
        """
        Fit the coefficients via your methods
        """

        # initiate the weight value and bias value
        self.w = np.zeros(self.dim)
        self.b = 0
        num = X.shape[0]

        # implement Gradient Descent for optimization
        for seq in range(self.max_iter):
            # gradients

            # for index,x_i in enumerate(self.X):
            #     if y[index] * (np.dot(x_i,self.w)-self.b) >= 1:
            #         dw = self.lambda_para*self.w
            #         db = 0
            #     else:
            #         dw = self.lambda_para*self.w - np.dot(x_i, [index])
            #         db = [index]
            condition = np.multiply(y,np.dot(X,self.w)+self.b)

            dw = self.w - self.C*np.dot(X[np.where(condition<1)].T,y[np.where(condition<1)])/num
            db = -self.C * np.sum(y[np.where(condition<1)])/num

            self.w = self.w - self.lr * dw
            self.b = self.b - self.lr * db

    def predict(self, X):
        """
        Use the trained model to generate prediction probabilities on a new
        collection of data points.
        """

        output = np.dot(X,self.w) + self.b
        predicted_label = np.sign(output)

        return predicted_label


# construct and train your models

In [30]:
# generate data
# X_data, y_data, mislabel = generate_data()
X_data, y_data, mislabel = generate_data(20,10000)
y_data = y_data.flatten()
print(mislabel)

0.0382


In [62]:
# generate data
# X_data, y_data, mislabel = generate_data()
X_data2, y_data2, mislabel2 = generate_data(30,15000)
y_data2 = y_data2.flatten()
print(mislabel2)

0.037


In [97]:
# generate data
# X_data, y_data, mislabel = generate_data()
X_data3, y_data3, mislabel3 = generate_data(10,5000)
y_data3 = y_data3.flatten()
print(mislabel3)

0.04


In [9]:
# constrcut model and train (remember record time)
def train1(X_train, y_train, dim, max_iter, C, tol, epsilon):
    model = SVM1(dim = dim, max_iter=max_iter, C=C,tol = tol, epsilon = epsilon)
    model.fit(X_train, y_train)
    return model

def train2(X_train, y_train, dim, lr, max_iter, C):
    model = SVM2(dim = dim, lr=lr, max_iter=max_iter, C = C)
    model.fit(X_train,y_train)
    return model

# predict and compare your results

In [10]:
# make prediction
def test(X_test,model):
    X_test_pred = model.predict(X_test)
    acc_test = np.sum(X_test_pred == y_test)/len(y_test)
    return acc_test

In [126]:
# split data
import time
from sklearn.svm import LinearSVC
# k_folds cross validate
k = 5
fold =int(len(X_data)/k)
acc_test_list1 = []
time_list1 = []
acc_test_list2 = []
time_list2 = []
acc_test_list3 = []
time_list3 = []

for i in range(k):
    X_train = np.r_[X_data[:i*fold],X_data[(i+1)*fold:]]
    y_train = np.r_[y_data[:i*fold],y_data[(i+1)*fold:]]
    X_test=X_data[i*fold:(i+1)*fold]
    y_test=y_data[i*fold:(i+1)*fold]

    # compared with answer
    print("\n第%d轮训练："%(i+1))
    print("SVM1:")
    t1 = time.perf_counter()
    model = train1(X_train,y_train,dim = 20, max_iter=100, C=1,tol = 1e-8, epsilon = 1e-8)
    t2 = time.perf_counter()
    t = t2-t1
    time_list1.append(t)
    acc_test1 = test(X_test, model)
    acc_test_list1.append(acc_test1)
    print("耗时：",t)
    print("测试集accuracy: ",acc_test1)

    print("SVM2:")
    t1 = time.perf_counter()
    model = train2(X_train,y_train,dim = 20, lr=0.01, max_iter=100, C = 1)
    t2 = time.perf_counter()
    t = t2-t1
    time_list2.append(t)
    acc_test2 = test(X_test, model)
    acc_test_list2.append(acc_test2)
    print("耗时：",t)
    print("测试集accuracy: ",acc_test2)

    print("sklearn:")
    t1 = time.perf_counter()
    clf = LinearSVC(penalty = 'l2', loss='hinge', max_iter=100000)
    clf.fit(X_train, y_train)
    t2 = time.perf_counter()
    t = t2-t1
    time_list3.append(t)
    acc_test3 = test(X_test, model)
    acc_test_list3.append(acc_test3)
    print("耗时：",t)
    print("测试集accuracy: ",acc_test3)



第1轮训练：
SVM1:
耗时： 43.60137220000615
测试集accuracy:  0.9515
SVM2:
耗时： 0.027651399999740534
测试集accuracy:  0.943
sklearn:


D:\Anaconda3\envs\ml\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


耗时： 1.4466724000085378
测试集accuracy:  0.943

第2轮训练：
SVM1:
耗时： 47.81860759999836
测试集accuracy:  0.948
SVM2:
耗时： 0.02711049999925308
测试集accuracy:  0.9495
sklearn:


D:\Anaconda3\envs\ml\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


耗时： 1.4882093999913195
测试集accuracy:  0.9495

第3轮训练：
SVM1:
耗时： 51.908864600001834
测试集accuracy:  0.947
SVM2:
耗时： 0.024727700001676567
测试集accuracy:  0.957
sklearn:


D:\Anaconda3\envs\ml\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


耗时： 1.46267780000926
测试集accuracy:  0.957

第4轮训练：
SVM1:
耗时： 54.914126599993324
测试集accuracy:  0.941
SVM2:
耗时： 0.026623400000971742
测试集accuracy:  0.949
sklearn:


D:\Anaconda3\envs\ml\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


耗时： 1.4921311999933096
测试集accuracy:  0.949

第5轮训练：
SVM1:
耗时： 59.3508620000066
测试集accuracy:  0.9585
SVM2:
耗时： 0.026343600009568036
测试集accuracy:  0.9555
sklearn:
耗时： 1.4843074999953387
测试集accuracy:  0.9555


D:\Anaconda3\envs\ml\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [127]:
# compare each methods

print("SVM1:")
print(acc_test_list1)
print("测试集平均accuracy: ",np.mean(acc_test_list1))
print(time_list1)
print("平均耗时：",np.mean(time_list1))

print("\nSVM2:")
print(acc_test_list2)
print("测试集平均accuracy: ",np.mean(acc_test_list2))
print(time_list2)
print("平均耗时：",np.mean(time_list2))

print("\nsklearn:")
print(acc_test_list3)
print("测试集平均accuracy: ",np.mean(acc_test_list3))
print(time_list3)
print("平均耗时：",np.mean(time_list3))

SVM1:
[0.9515, 0.948, 0.947, 0.941, 0.9585]
测试集平均accuracy:  0.9491999999999999
[43.60137220000615, 47.81860759999836, 51.908864600001834, 54.914126599993324, 59.3508620000066]
平均耗时： 51.51876660000126

SVM2:
[0.943, 0.9495, 0.957, 0.949, 0.9555]
测试集平均accuracy:  0.9507999999999999
[0.027651399999740534, 0.02711049999925308, 0.024727700001676567, 0.026623400000971742, 0.026343600009568036]
平均耗时： 0.02649132000224199

sklearn:
[0.943, 0.9495, 0.957, 0.949, 0.9555]
测试集平均accuracy:  0.9507999999999999
[1.4466724000085378, 1.4882093999913195, 1.46267780000926, 1.4921311999933096, 1.4843074999953387]
平均耗时： 1.4747996599995532


In [63]:
# split data
import time
from sklearn.svm import LinearSVC
# k_folds cross validate
k = 5
fold =int(len(X_data2)/k)
acc_test_list11 = []
time_list11 = []
acc_test_list22 = []
time_list22 = []
acc_test_list33 = []
time_list33 = []

for i in range(k):
    X_train = np.r_[X_data2[:i*fold],X_data2[(i+1)*fold:]]
    y_train = np.r_[y_data2[:i*fold],y_data2[(i+1)*fold:]]
    X_test=X_data2[i*fold:(i+1)*fold]
    y_test=y_data2[i*fold:(i+1)*fold]

    # compared with answer
    print("\n第%d轮训练："%(i+1))
    print("SVM1:")
    t1 = time.perf_counter()
    model = train1(X_train,y_train, dim = 30, max_iter=100, C=1,tol = 1e-8, epsilon = 1e-8)
    t2 = time.perf_counter()
    t = t2-t1
    time_list11.append(t)
    acc_test1 = test(X_test, model)
    acc_test_list11.append(acc_test1)
    print("耗时：",t)
    print("测试集accuracy: ",acc_test1)

    print("SVM2:")
    t1 = time.perf_counter()
    model = train2(X_train,y_train,dim = 30, lr=0.01, max_iter=100, C = 1)
    t2 = time.perf_counter()
    t = t2-t1
    time_list22.append(t)
    acc_test2 = test(X_test, model)
    acc_test_list22.append(acc_test2)
    print("耗时：",t)
    print("测试集accuracy: ",acc_test2)

    print("sklearn:")
    t1 = time.perf_counter()
    clf = LinearSVC(penalty = 'l2', loss='hinge', max_iter=100000)
    clf.fit(X_train, y_train)
    t2 = time.perf_counter()
    t = t2-t1
    time_list33.append(t)
    acc_test3 = test(X_test, model)
    acc_test_list33.append(acc_test3)
    print("耗时：",t)
    print("测试集accuracy: ",acc_test3)


第1轮训练：
SVM1:
耗时： 159.49881640000967
测试集accuracy:  0.9536666666666667
SVM2:
耗时： 0.0462198999885004
测试集accuracy:  0.956
sklearn:


D:\Anaconda3\envs\ml\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


耗时： 3.7866291999816895
测试集accuracy:  0.956

第2轮训练：
SVM1:
耗时： 187.79496260001906
测试集accuracy:  0.954
SVM2:
耗时： 0.046176400006515905
测试集accuracy:  0.9513333333333334
sklearn:


D:\Anaconda3\envs\ml\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


耗时： 3.889095799997449
测试集accuracy:  0.9513333333333334

第3轮训练：
SVM1:
耗时： 183.29716719998396
测试集accuracy:  0.9536666666666667
SVM2:
耗时： 0.050320300011662766
测试集accuracy:  0.95
sklearn:


D:\Anaconda3\envs\ml\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


耗时： 3.8688754999893717
测试集accuracy:  0.95

第4轮训练：
SVM1:
耗时： 178.31898399998317
测试集accuracy:  0.9533333333333334
SVM2:
耗时： 0.057560899993404746
测试集accuracy:  0.9513333333333334
sklearn:


D:\Anaconda3\envs\ml\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


耗时： 3.8002525999909267
测试集accuracy:  0.9513333333333334

第5轮训练：
SVM1:
耗时： 191.07833839999512
测试集accuracy:  0.9506666666666667
SVM2:
耗时： 0.046223699988331646
测试集accuracy:  0.9553333333333334
sklearn:
耗时： 3.840269000007538
测试集accuracy:  0.9553333333333334


D:\Anaconda3\envs\ml\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [64]:
# compare each methods

print("SVM1:")
print(acc_test_list11)
print("测试集平均accuracy: ",np.mean(acc_test_list11))
print(time_list11)
print("平均耗时：",np.mean(time_list11))

print("\nSVM2:")
print(acc_test_list22)
print("测试集平均accuracy: ",np.mean(acc_test_list22))
print(time_list22)
print("平均耗时：",np.mean(time_list22))

print("\nsklearn:")
print(acc_test_list33)
print("测试集平均accuracy: ",np.mean(acc_test_list33))
print(time_list33)
print("平均耗时：",np.mean(time_list33))

SVM1:
[0.9536666666666667, 0.954, 0.9536666666666667, 0.9533333333333334, 0.9506666666666667]
测试集平均accuracy:  0.9530666666666667
[159.49881640000967, 187.79496260001906, 183.29716719998396, 178.31898399998317, 191.07833839999512]
平均耗时： 179.9976537199982

SVM2:
[0.956, 0.9513333333333334, 0.95, 0.9513333333333334, 0.9553333333333334]
测试集平均accuracy:  0.9527999999999999
[0.0462198999885004, 0.046176400006515905, 0.050320300011662766, 0.057560899993404746, 0.046223699988331646]
平均耗时： 0.04930023999768309

sklearn:
[0.956, 0.9513333333333334, 0.95, 0.9513333333333334, 0.9553333333333334]
测试集平均accuracy:  0.9527999999999999
[3.7866291999816895, 3.889095799997449, 3.8688754999893717, 3.8002525999909267, 3.840269000007538]
平均耗时： 3.837024419993395


In [128]:
# split data
import time
from sklearn.svm import LinearSVC
# k_folds cross validate
k = 5
fold =int(len(X_data3)/k)
acc_test_list111 = []
time_list111 = []
acc_test_list222 = []
time_list222 = []
acc_test_list333 = []
time_list333 = []

for i in range(k):
    X_train = np.r_[X_data3[:i*fold],X_data3[(i+1)*fold:]]
    y_train = np.r_[y_data3[:i*fold],y_data3[(i+1)*fold:]]
    X_test=X_data3[i*fold:(i+1)*fold]
    y_test=y_data3[i*fold:(i+1)*fold]
    # compared with answer
    print("\n第%d轮训练："%(i+1))
    print("SVM1:")
    t1 = time.perf_counter()
    model = train1(X_train,y_train, dim = 10, max_iter=100, C=1,tol = 1e-8, epsilon = 1e-8)
    t2 = time.perf_counter()
    t = t2-t1
    time_list111.append(t)
    acc_test1 = test(X_test, model)
    acc_test_list111.append(acc_test1)
    print("耗时：",t)
    print("测试集accuracy: ",acc_test1)

    print("SVM2:")
    t1 = time.perf_counter()
    model = train2(X_train,y_train,dim = 10, lr=0.01, max_iter=100, C = 1)
    t2 = time.perf_counter()
    t = t2-t1
    time_list222.append(t)
    acc_test2 = test(X_test, model)
    acc_test_list222.append(acc_test2)
    print("耗时：",t)
    print("测试集accuracy: ",acc_test2)

    print("sklearn:")
    t1 = time.perf_counter()
    clf = LinearSVC(penalty = 'l2', loss='hinge', max_iter=100000)
    clf.fit(X_train, y_train)
    t2 = time.perf_counter()
    t = t2-t1
    time_list333.append(t)
    acc_test3 = test(X_test, model)
    acc_test_list333.append(acc_test3)
    print("耗时：",t)
    print("测试集accuracy: ",acc_test3)


第1轮训练：
SVM1:
耗时： 18.901264000000083
测试集accuracy:  0.944
SVM2:
耗时： 0.012744399995426647
测试集accuracy:  0.945
sklearn:


D:\Anaconda3\envs\ml\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


耗时： 0.2769842000125209
测试集accuracy:  0.945

第2轮训练：
SVM1:
耗时： 18.753948799989303
测试集accuracy:  0.951
SVM2:
耗时： 0.0126419000007445
测试集accuracy:  0.954
sklearn:


D:\Anaconda3\envs\ml\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


耗时： 0.2531458000012208
测试集accuracy:  0.954

第3轮训练：
SVM1:
耗时： 19.18545720000111
测试集accuracy:  0.956
SVM2:
耗时： 0.012590400001499802
测试集accuracy:  0.957
sklearn:


D:\Anaconda3\envs\ml\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


耗时： 0.2692286000092281
测试集accuracy:  0.957

第4轮训练：
SVM1:
耗时： 20.798065799986944
测试集accuracy:  0.948
SVM2:
耗时： 0.014343699993332848
测试集accuracy:  0.949
sklearn:


D:\Anaconda3\envs\ml\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


耗时： 0.26876520000223536
测试集accuracy:  0.949

第5轮训练：
SVM1:
耗时： 20.169730000008713
测试集accuracy:  0.945
SVM2:
耗时： 0.009860899997875094
测试集accuracy:  0.945
sklearn:
耗时： 0.23756069999944884
测试集accuracy:  0.945


D:\Anaconda3\envs\ml\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [129]:
# compare each methods

print("SVM1:")
print(acc_test_list111)
print("测试集平均accuracy: ",np.mean(acc_test_list111))
print(time_list111)
print("平均耗时：",np.mean(time_list111))

print("\nSVM2:")
print(acc_test_list222)
print("测试集平均accuracy: ",np.mean(acc_test_list222))
print(time_list222)
print("平均耗时：",np.mean(time_list222))

print("\nsklearn:")
print(acc_test_list333)
print("测试集平均accuracy: ",np.mean(acc_test_list333))
print(time_list333)
print("平均耗时：",np.mean(time_list333))

SVM1:
[0.944, 0.951, 0.956, 0.948, 0.945]
测试集平均accuracy:  0.9488
[18.901264000000083, 18.753948799989303, 19.18545720000111, 20.798065799986944, 20.169730000008713]
平均耗时： 19.56169315999723

SVM2:
[0.945, 0.954, 0.957, 0.949, 0.945]
测试集平均accuracy:  0.95
[0.012744399995426647, 0.0126419000007445, 0.012590400001499802, 0.014343699993332848, 0.009860899997875094]
平均耗时： 0.012436259997775779

sklearn:
[0.945, 0.954, 0.957, 0.949, 0.945]
测试集平均accuracy:  0.95
[0.2769842000125209, 0.2531458000012208, 0.2692286000092281, 0.26876520000223536, 0.23756069999944884]
平均耗时： 0.2611369000049308
